In [1]:
from datasets import concatenate_datasets
from datasets import load_dataset
import torch
import json
from dataclasses import dataclass, field
import re
from typing import Optional
import os
import evaluate
import torch
from datasets import concatenate_datasets
from datasets import load_dataset
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from datasets import load_from_disk
import pickle
import pandas as pd
import random
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed
from trl.core import LengthSampler
tqdm.pandas()

In [2]:
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
toxicity = evaluate.load("ybelkada/toxicity", "DaNLP/da-electra-hatespeech-detection", module_type="measurement")
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, cache_dir='/home/chenboc1/localscratch2/chenboc1/trl/.cache').to(device)


In [3]:
FULL_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/result.json'
JOE_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/prompt_selected/Joe_Biden_response.json'
IPHONE_RESULT = '/home/chenboc1/localscratch2/chenboc1/trl/examples/create_prompts/data/prompt_selected/Iphone_response.json'

In [4]:

with open(FULL_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    FULL_RESULT = json.load(json_file)

with open(JOE_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    JOE_RESULT = json.load(json_file)

with open(IPHONE_RESULT, "r") as json_file:
    # Load the JSON data into a Python dictionary
    IPHONE_RESULT = json.load(json_file)


In [5]:
FULL_RESULT.keys()

dict_keys(['ctx', 'real'])

In [6]:
JOE_RESULT.keys()

dict_keys(['ctx_Joe_Biden', 'realtox_Joe_Biden'])

In [7]:
IPHONE_RESULT.keys()

dict_keys(['ctx_Joe_Biden', 'realtox_Joe_Biden'])

In [12]:
IPHONE_RESULT['ctx_Joe_Biden'].keys()

dict_keys(['prompt', 'gpt_generation', 'DaNLP_score', 'roberta_score', 'DaNLP_score_Gen', 'roberta_score_Gen'])

In [13]:
IPHONE_RESULT['realtox_Joe_Biden'].keys()

dict_keys(['prompt', 'gpt_generation', 'DaNLP_score', 'roberta_score', 'DaNLP_score_Gen', 'roberta_score_Gen'])

In [9]:
IPHONE_RESULT['ctx_Joe_Biden'].update({'DaNLP_score_Gen':[], 'roberta_score_Gen': []})
IPHONE_RESULT['realtox_Joe_Biden'].update({'DaNLP_score_Gen':[], 'roberta_score_Gen': []})

In [10]:
IPHONE_RESULT['realtox_Joe_Biden']['DaNLP_score_Gen']

[]

In [11]:
IPHONE_RESULT['realtox_Joe_Biden'].keys()

dict_keys(['prompt', 'gpt_generation', 'DaNLP_score', 'roberta_score', 'DaNLP_score_Gen', 'roberta_score_Gen'])

In [34]:
response_column = IPHONE_RESULT['ctx_Joe_Biden']['gpt_generation']
input_texts = []
BATCH_SIZE = 256
for i, example in enumerate(response_column):
    input_text = example
    input_texts.append(input_text[:2000])

    if (i + 1) % BATCH_SIZE == 0:
        toxicity_score = toxicity.compute(predictions=input_texts)['toxicity']
        IPHONE_RESULT['ctx_Joe_Biden']['DaNLP_score_Gen'].extend(toxicity_score)
        toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(
            device
        )
        logits = toxicity_model(**toxicity_inputs).logits.float()
        toxicity_labels = (logits[:, 0]).tolist()        

        IPHONE_RESULT['ctx_Joe_Biden']['roberta_score_Gen'].extend(toxicity_labels)        
        input_texts = []
    if (i + 1) % (BATCH_SIZE*5) == 0:
        print("here is epoch i")



here is epoch i
here is epoch i


In [ ]:
response_column = IPHONE_RESULT['realtox_Joe_Biden']['gpt_generation']
input_texts = []
for i, example in enumerate(response_column):
    input_text = example
    input_texts.append(input_text[:2000])

    if (i + 1) % BATCH_SIZE == 0:
        toxicity_score = toxicity.compute(predictions=input_texts)['toxicity']
        IPHONE_RESULT['realtox_Joe_Biden']['DaNLP_score_Gen'].extend(toxicity_score)
        toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(
            device
        )
        logits = toxicity_model(**toxicity_inputs).logits.float()
        toxicity_labels = (logits[:, 0]).tolist()        

        IPHONE_RESULT['realtox_Joe_Biden']['roberta_score_Gen'].extend(toxicity_labels)        
        input_texts = []
    if (i + 1) % (BATCH_SIZE*5) == 0:
        print("here is epoch i")



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   │   toxicity_inputs = toxicity_tokenizer(input_texts, padding=True, truncation=True,    │
│   11 │   │   │   device                                                                          │
│   12 │   │   )                                                                                   │
│ ❱ 13 │   │   logits = toxicity_model(**toxicity_inputs).logits.float()                           │
│   14 │   │   toxicity_labels = (logits[:, 0]).tolist()                                           │
│   15 │   │                                                                                       │
│   16 │   │   IPHONE_RESULT['realtox_Joe_Biden']['roberta_score_Gen'].extend(toxicity_labels)     │
│                                                                                                  │
│ /localscratch/chenboc1/anaconda3/envs/trl/lib/python3.11/site-packages/torch/nn/modules/module.p │
│ y:1501 in _call_impl                                                                             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /localscratch/chenboc1/anaconda3/envs/trl/lib/python3.11/site-packages/transformers/models/rober │
│ ta/modeling_roberta.py:1216 in forward                                                           │
│                                                                                                  │
│   1213 │   │   """                                                                               │
│   1214 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1215 │   │                                                                                     │
│ ❱ 1216 │   │   outputs = self.roberta(                                                           │
│   1217 │   │   │   input_ids,                                                                    │
│   1218 │   │   │   attention_mask=attention_mask,                                                │
│   1219 │   │   │   token_type_ids=token_type_ids,                                                │
│                                                                                                  │
│ /localscratch/chenboc1/anaconda3/envs/trl/lib/python3.11/site-packages/torch/nn/modules/module.p │
│ y:1501 in _call_impl                                                                             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [ ]:
with open('realtox_Joe_Biden.json', 'w') as json_file:
    # Load the JSON data into a Python dictionary
    json.dump(realtox_Joe_Biden, json_file)